In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None


In [2]:
df = pd.read_csv('events_up_to_01062018.csv', low_memory = False)

In [3]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_model = df.groupby('person')['model'].apply(list).reset_index()

In [14]:
df['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [60]:
df_feat = pd.DataFrame(data = person)

In [46]:
df_model.head()

,person,model
0,0008ed71,"[LG G4 H818P, Samsung Galaxy S6 Flat, iPhone S..."
1,00091926,"[iPhone 6 Plus, iPhone 6S, iPhone 6S, Motorola..."
2,00091a7a,"[iPhone SE, iPhone 6, nan, iPhone 6S, nan, nan..."
3,000ba417,"[Samsung Galaxy A3 2016, Samsung Galaxy Gran P..."
4,000c79fe,"[iPhone 7, nan, iPhone 7, nan, nan, nan, nan, ..."


In [61]:
events = ['viewed product', 'brand listing', 'visited site', 'ad campaign hit', 'generic listing','searched products','search engine hit','checkout','staticpage','conversion','lead']
df_events=[None]*11

In [62]:
for x in range(11):
    df_events[x]= df.loc[df['event'] == events[x]]
    (df_events[x])[events[x]] = 1
    df_events[x] = df_events[x].groupby('person').agg({events[x]:'sum'}).reset_index()
    

In [63]:
for x in range(11):
    df_feat = df_feat.merge(df_events[x], left_on = 'person', right_on = 'person', how = 'left')

In [64]:
df_feat.head()

,person,viewed product,brand listing,visited site,ad campaign hit,generic listing,searched products,search engine hit,checkout,staticpage,conversion,lead
0,4886f805,4.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN
1,ad93850f,20.0,15.0,5.0,10.0,7.0,NaN,7.0,1.0,NaN,NaN,NaN
2,0297fc1e,404.0,4.0,95.0,29.0,21.0,6.0,NaN,7.0,NaN,NaN,1.0
3,2d681dd8,13.0,5.0,2.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN
4,cccea85e,739.0,7.0,22.0,15.0,20.0,1.0,26.0,1.0,5.0,NaN,NaN


In [131]:
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)
df_train = df_labels.merge(df_feat, left_on = 'person', right_on = 'person', how = 'left')
df_train.head()

,person,label,viewed product,brand listing,visited site,ad campaign hit,generic listing,searched products,search engine hit,checkout,staticpage,conversion,lead
0,0566e9c1,0,23.0,3.0,17.0,6.0,15.0,NaN,1.0,1.0,1.0,1.0,NaN
1,6ec7ee77,0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abe7a2fb,0,31.0,14.0,22.0,9.0,9.0,6.0,4.0,1.0,NaN,NaN,NaN
3,34728364,0,24.0,5.0,4.0,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN
4,87ed62de,0,9.0,NaN,1.0,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


In [66]:
X, y = df_train.iloc[:,2:13],df_train.iloc[:,1]
X.head()

,viewed product,brand listing,visited site,ad campaign hit,generic listing,searched products,search engine hit,checkout,staticpage,conversion,lead
0,23.0,3.0,17.0,6.0,15.0,NaN,1.0,1.0,1.0,1.0,NaN
1,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31.0,14.0,22.0,9.0,9.0,6.0,4.0,1.0,NaN,NaN,NaN
3,24.0,5.0,4.0,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN
4,9.0,NaN,1.0,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


In [135]:

y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [136]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [141]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 50, alpha = 10, n_estimators = 10)

In [142]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)



In [143]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=50, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [144]:
persons = df_labels['person']
df_predict = (df_feat.loc[~df_feat.person.isin(persons)])
X_predict = df_predict.drop('person', axis=1)
df_predict.shape

(19415, 12)

In [145]:
preds = xg_reg.predict(X_test)

In [146]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.266565


In [147]:
my_classifier1 = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [148]:
my_classifier1.fit(X_train,y_train)
predictions=my_classifier1.predict(X_test)

In [82]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

0.796138388932062

In [83]:
entrie = xg_reg.predict(X_predict)

In [84]:
entrie

array([0.19728023, 0.22467422, 0.20149899, ..., 0.2089524 , 0.2089524 ,
       0.2089524 ], dtype=float32)

In [85]:
seriesita = pd.Series(entrie)

In [114]:
df_entrie = df_predict['person'].to_frame()
df_entrie['label'] = seriesita

In [115]:
df_entrie.head()

,person,label
0,4886f805,0.197280
2,0297fc1e,0.201499
3,2d681dd8,0.224972
4,cccea85e,0.222796
5,4c8a8b93,0.206293


In [126]:
df_csv = pd.read_csv('trocafone_kaggle_test.csv', low_memory = False)

In [127]:
df_csv = df_csv.merge(df_entrie, left_on='person' , right_on='person' , how='left')
df_csv.head()

,person,label
0,4886f805,0.197280
1,0297fc1e,0.201499
2,2d681dd8,0.224972
3,cccea85e,0.222796
4,4c8a8b93,0.206293


In [149]:
df_csv['label'] = df_csv['label'].fillna(0)
df_csv[df_csv['label'] < 0] = 0


In [130]:
df_csv.to_csv(path_or_buf = 'entrie', index = False)